# Developer Section

In [1]:
# define rooms and items

#furniture
couch = {
    "name": "couch",
    "type": "furniture",
}


piano = {
    "name": "piano",
    "type": "furniture",
}
# Added for mini escape room start

dresser={
    "name": "dresser",
    "type": "furniture",
}

queen_bed={
    "name": "queen bed",
    "type": "furniture",
}

double_bed={
    "name": "double bed",
    "type": "furniture",
}

dinning_table={
    "name": "dinning table",
    "type": "furniture",
}
# doors

door_a = {
    "name": "door a",
    "type": "door",
}

# Added for mini escape room start
door_b = {
    "name": "door b",
    "type": "door",
}

door_c = {
    "name": "door c",
    "type": "door",
}
door_d = {
    "name": "door d",
    "type": "door",
}

#keys

key_a = {
    "name": "key for door a",
    "type": "key",
    "target": door_a,
}

# Added for mini escape room start

key_b = {
    "name": "key for door b",
    "type": "key",
    "target": door_b,
}
key_c = {
    "name": "key for door c",
    "type": "key",
    "target": door_c,
}
key_d = {
    "name": "key for door d",
    "type": "key",
    "target": door_d,
}

# rooms

game_room = {
    "name": "game room",
    "type": "room",
}

outside = {
  "name": "outside"
}


# Added for mini escape room start
bedroom_room1 = {
    "name": "bedroom room 1",
    "type": "room",
}

bedroom_room2 = {
    "name": "bedroom room 2",
    "type": "room",
}

living_room = {
    "name": "living room",
    "type": "room",
}



all_rooms = [game_room, outside , bedroom_room1, bedroom_room2 ,living_room]

all_doors = [door_a , door_b , door_c , door_d]

# define which items/rooms are related

object_relations = {
    "game room": [couch, piano, door_a],
    "piano": [key_a],
    #change below for new implementation
    "outside": [door_a],
    "door a": [game_room,bedroom_room1],
    "bedroom room 1":[queen_bed,door_c,door_b,door_a],
    "queen bed":[key_b],
    "door b":[bedroom_room1,bedroom_room2],
    "bedroom room 2":[double_bed,dresser,door_b],
    "double bed":[key_c],
    "dresser":[key_d],
    "living room":[dinning_table,door_d,door_c],
    "door c":[bedroom_room1,living_room],
    "door d":[outside]
}

# define game state. Do not directly change this dict.
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "target_room": outside
}

In [3]:
def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def start_game():
    """
    Start the game
    """
    print("You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!")
    play_room(game_state["current_room"])

def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either
    explore (list all items in this room) or examine an item found here.
    """
    game_state["current_room"] = room
    if(game_state["current_room"] == game_state["target_room"]):
        print("Congrats! You escaped the room!")
    else:
        print("You are now in " + room["name"])
        intended_action = input("What would you like to do? Type 'explore' or 'examine'? ").strip()
        if intended_action.lower() == "explore":  #Added lower method debug
            explore_room(room)
            play_room(room)
        elif intended_action.lower() == "examine": #Added lower method debug
            examine_item(input("What would you like to examine? ").strip().lower()) #Added tp debug
        else:
            print("Not sure what you mean. Type 'explore' or 'examine'.")
            play_room(room)
        linebreak()

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print("You explore the room. This is " + room["name"] + ". You find " + ", ".join(items))

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
   # print(f"Connected_rooms:" {connected_rooms}) #added for debug
    for room in connected_rooms:
        if(not current_room == room):
            return room

def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None

    for item in object_relations[current_room["name"]]:
        if(item["name"] == item_name):
            output = "You examine " + item_name + ". "
            if(item["type"] == "door"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        have_key = True
                if(have_key):
                    output += "You unlock it with a key you have."
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "It is locked but you don't have the key."
            else:
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "You find " + item_found["name"] + "."
                else:
                    output += "There isn't anything interesting about it."
            print(output)
            break

    if(output is None):
        print("The item you requested is not found in the current room.")

    if(next_room and input("Do you want to go to the next room? Enter 'yes' or 'no' ").strip().lower() == 'yes'):
        play_room(next_room)
    else:
        play_room(current_room)


In [ ]:
game_state = INIT_GAME_STATE.copy()

start_game()

You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!
You are now in game room
What would you like to do? Type 'explore' or 'examine'? explore
You explore the room. This is game room. You find couch, piano, door a
You are now in game room
What would you like to do? Type 'explore' or 'examine'? explore
You explore the room. This is game room. You find couch, piano, door a
You are now in game room
What would you like to do? Type 'explore' or 'examine'? examine
What would you like to examine? piano
You examine piano. You find key for door a.
You are now in game room
What would you like to do? Type 'explore' or 'examine'? examine
What would you like to examine? door a
You examine door a. You unlock it with a key you have.
Do you want to go to the next room? Enter 'yes' or 'no' yes
You

# Research Section